In [51]:
import torch
import torchvision.models as models
import os
import csv
import torch.nn as nn
import torch.nn.functional as F
import copy
import pandas as pd
from resnet20 import ResNet20

In [52]:

def fold_batch_norm(model: nn.Module, data_path):
    """
    Fold BatchNorm layers into Conv layers, then save the weights and biases
    of each layer into separate CSV files in the specified `data_path`.
    """
    if not os.path.exists(data_path):
        os.makedirs(data_path)  # Create the directory if it doesn't exist

    conv_module = None
    bn_next = False

    model.eval()

    for index, (name, module) in enumerate(model.named_modules()):
        # Identify Conv2d layers and handle BatchNorm folding
        if isinstance(module, nn.Conv2d):
            bn_next = True
            conv_module = module
        if isinstance(module, nn.BatchNorm2d) and bn_next:
            bn_next = False

            gamma = module.weight.data  # The scale factor
            beta = module.bias.data  # The shift factor
            mean = module.running_mean  # The moving average of means
            var = module.running_var  # The moving average of variances
            epsilon = module.eps  # Epsilon to avoid division by zero

            # Compute the new Conv2d weights (fold BatchNorm into Conv)
            with torch.no_grad():
                # Scale Conv weights by gamma / sqrt(var + epsilon)
                conv_module.weight.data = conv_module.weight.data * (gamma / torch.sqrt(var + epsilon)).view(-1, 1, 1, 1)

                if ".bn" in name:
                    name_to_save = name.replace(".bn", "_conv")
                elif ".shortcut_bn" in name:  # Handling shortcut BatchNorm
                    name_to_save = name.replace(".shortcut_bn", "_shortcut")
                     # If there is a stride (downsampling), scale the weights and biases accordingly
                    # if conv_module.stride != (1, 1):  # Check if stride > 1 (downsampling)
                        # When stride > 1, we need to scale the gamma and beta by the stride factor
                        # scale_factor = torch.sqrt(torch.tensor(1.0 / (conv_module.stride[0] ** 2)))
                        # conv_module.weight.data *= scale_factor.view(-1, 1, 1, 1)
                        # gamma *= scale_factor
                        # beta *= scale_factor
                    # print(f"Processing shortcut BN: {name_to_save}")
                elif "bn1" in name:
                    name_to_save = name.replace("bn1", "layer0_conv1")

                # Save weights
                weights_filename = os.path.join(data_path, f'{name_to_save}_weight.csv')
                print(f'weight name: {weights_filename}')
                clean_data = conv_module.weight.data.numpy() if hasattr(conv_module.weight.data, 'numpy') else conv_module.weight.data
                with open(weights_filename, mode='w', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerows(clean_data.flatten().reshape(1, -1))

                # Adjust Conv bias using BatchNorm parameters
                if conv_module.bias is not None:
                    conv_module.bias.data = (gamma * (conv_module.bias.data - mean) / torch.sqrt(var + epsilon)) + beta
                    
                    # Save bias
                    biases_filename = os.path.join(data_path, f'{name_to_save}_bias.csv')
                    print(f'bias name: {biases_filename}')
                    clean_bias_data = conv_module.bias.numpy() if hasattr(conv_module.bias, 'numpy') else conv_module.bias
                    with open(biases_filename, mode='w', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerows(clean_bias_data.flatten().reshape(1, -1))
                else:
                    # If Conv2d doesn't have a bias, add one using BatchNorm parameters
                    biases_filename = os.path.join(data_path, f'{name_to_save}_bias.csv')
                    print(f'bias name: {biases_filename}')
                    conv_module.bias = nn.Parameter((gamma * (-mean) / torch.sqrt(var + epsilon)) + beta)
                    clean_bias_data = conv_module.bias.numpy() if hasattr(conv_module.bias, 'numpy') else conv_module.bias
                    with open(biases_filename, mode='w', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerows(clean_bias_data.flatten().reshape(1, -1))
            # Reset conv_module after processing BatchNorm folding
            conv_module = None

        # Handle fully connected layer (fc)
        if name == "fc":
            fc_weights_filename = os.path.join(data_path, f'layer_fc_weight.csv')
            fc_bias_filename = os.path.join(data_path, f'layer_fc_bias.csv')
            print(f'fc layer weights name: {fc_weights_filename}')
            print(f'fc layer bias name: {fc_bias_filename}')
            clean_weights_data = module.weight.data.numpy() if hasattr(module.weight.data, 'numpy') else module.weight.data
            clean_bias_data = module.bias.data.numpy() if hasattr(module.bias.data, 'numpy') else module.bias.data
            with open(fc_weights_filename, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerows(clean_weights_data.flatten().reshape(1, -1))
            
            with open(fc_bias_filename, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerows(clean_bias_data.flatten().reshape(1, -1))

    print("BatchNorm folded into Conv layers and weights/biases saved to CSV.")
    return replace_bn_with_identity(model)

def replace_bn_with_identity(model):
    """
    Replaces all BatchNorm layers in the model with Identity layers.
    This is done after folding BatchNorm into Conv layers.
    """
    for name, module in model.named_children():
        if isinstance(module, nn.BatchNorm2d) or isinstance(module, nn.BatchNorm1d) or isinstance(module, nn.BatchNorm3d):
            # Replace with Identity layer
            setattr(model, name, nn.Identity())
        else:
            # Recursively handle nested submodules
            replace_bn_with_identity(module)
    return model

In [ ]:
# WEIGHTS_DIR = './../models/resnet20_2025-01-23.pth'
data_path = './../../weights/resnet20/'

# load weights
channel_values = [16, 32, 64]
num_classes = 10
fake_data = torch.rand((1, 3, 32, 32))
model = ResNet20(channel_values, num_classes)

model.load_state_dict(torch.load('./../models/resnet20_2025-01-26.pth'), strict=False)
model.eval()
print(model(fake_data))
model = fold_batch_norm(model, data_path)
print(model(fake_data))

tensor([[  1.1213,  -6.7830,   6.1559,   4.1597,   3.2178,  -8.5106,   8.5623,
         -10.8796,  -5.8799,  -4.5208]], grad_fn=<AddmmBackward0>)
weight name: ./../../weights/resnet20/layer0_conv1_weight.csv
bias name: ./../../weights/resnet20/layer0_conv1_bias.csv
weight name: ./../../weights/resnet20/layer1_block1_conv1_weight.csv
bias name: ./../../weights/resnet20/layer1_block1_conv1_bias.csv
weight name: ./../../weights/resnet20/layer1_block1_conv2_weight.csv
bias name: ./../../weights/resnet20/layer1_block1_conv2_bias.csv
weight name: ./../../weights/resnet20/layer1_block2_conv1_weight.csv
bias name: ./../../weights/resnet20/layer1_block2_conv1_bias.csv
weight name: ./../../weights/resnet20/layer1_block2_conv2_weight.csv
bias name: ./../../weights/resnet20/layer1_block2_conv2_bias.csv
weight name: ./../../weights/resnet20/layer1_block3_conv1_weight.csv
bias name: ./../../weights/resnet20/layer1_block3_conv1_bias.csv
weight name: ./../../weights/resnet20/layer1_block3_conv2_weight

/tmp/ipykernel_364569/1172442577.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./../models/resnet20_2025-01-23.pth'), strict=False)


In [54]:
# # Initialize index
# i = 0
# data_path ="./../../weights/vgg11/"

# # Iterate over each layer in weights
# for layer in weights.keys():
#     # Check if the layer is a bias
#     data = weights[layer].numpy() if hasattr(weights[layer], 'numpy') else weights[layer]
#     if 'bias' in layer:
#         # Define CSV file name for biases
#         filename = f'{data_path}{layers[i]}_bias.csv'
#         # Open the file for writing
#         with open(filename, mode='w', newline='') as file:
#             writer = csv.writer(file)
#             # Write the shape of the bias
#             # writer.writerow([f'{layers[i]} bias shape:', data.shape])
#             # Write the bias values
#             # writer.writerow([f'{layers[i]} bias:'])
#             writer.writerows(data.flatten().reshape(1, -1))  # Flatten and reshape to write as single row
#         i += 1
#     else:
#         # Define CSV file name for weights
#         filename = f'{data_path}{layers[i]}_weights.csv'
#         # Open the file for writing
#         with open(filename, mode='w', newline='') as file:
#             writer = csv.writer(file)
#             # Write the shape of the weights
#             # writer.writerow([f'{layers[i]} weights shape:', data.shape])
#             # Write the weights values
#             # writer.writerow([f'{layers[i]} weights:'])
#             writer.writerows(data.flatten().reshape(1, -1))  # Flatten and reshape to write as single row

# print("All Layers Exported")